# Creating a repository from DMTF Schema

In addition to creating a repository with specifically defined classes, qualifier declarations and instances the repository can be created using the published DMTF schema as the source for qualifier declarations and classes.  The DMTF schema is released on a regular basis with new functionality, fixes, etc. And is published on the DMTF web site. It includes the set of CIM qualifier declarations defined by the DMTF and the CIM classes that make up the CIM model. It is available in both MOF and XML formats.  The method documented here uses the MOF files as the basis for creating CIM qualifier declarations and CIM classes in the mock repository.

Using this method requires that you make the following decisions:

1. Which version of the published DMTF released schema you intend to use.

2. Where in your local environment do you want to save and expand the schema. This should be a directory and serves both as the storage space for download and expansion and the location checked before download so that the schema is not downloaded each time it is used.

3. Use of the experimental schema for the defined DMTF schema release as well as the released schema.

4. The leaf classes you need. The method takes responsibility for determining superclasses and other required classes ( ex. associations, embedded instances) defined in the mof for any of the leaf classes you specify.

The following example defines a DMTF schema version, the destination directory, and the leaf classes of interest and then calls the compile_dmtf_schema method to create a mock repository. At the end of the operation, the mock repository is includes the complete DMTF set of qualifier declarations (there are only about 100 of them) and the classes you specified and their dependencies.  The destination directory should also include the complete download from the DMTF and the expanded mof files for the qualifier declarations and classes.

Note: that each time you call compile_dmtf_schema he installs the complete set of qualifier declarations in the mock repository so that it is best to create a complete list of leaf classes required an call this method only once.

Warning: Some of these cells create new objects on the server. Once those objects are created, they cannot simply be
recreated on the server so the repeated execution of some of the cells without restarting (see Kernel in the menu) will
cause exception returns from the server.


In [ ]:
import pywbem
import pywbem_mock

import os
print(os.getcwd())

# Defines the version of the DMTF schema to be installed.
# This demo is done with the schema already loaded into the pywbem development environment assuming that
# the user is using the docs/notebooks directory. This may change as pywbem updates the schema use
# in out tests
DMTF_TEST_SCHEME_VER = (2, 49, 0)
# location of schema 2.49.0 in pywbem 0.13.0 development
# code cloed from github
SCHEMA_DIR = os.path.join('..', '..', 'tests', 'schema')
print(SCHEMA_DIR)

# An alternative would be to define your own schema and schema directory
#DMTF_TEST_SCHEME_VER = "2.51.0"
#SCHEMA_DIR = "."

classnames = ['CIM_RegisteredProfile', 'CIM_Namespace', 'CIM_ObjectManager',
              'CIM_ElementConformsToProfile', 'CIM_ReferencedProfile']

conn = pywbem_mock.FakedWBEMConnection(default_namespace='root/cimv2')
print(conn)

conn.compile_dmtf_schema(DMTF_TEST_SCHEME_VER, SCHEMA_DIR,
                                 class_names=classnames, verbose=False)
conn.display_repository()

## Add some more classes to this repository

Subsequent to the installation of the DMTF schema we can add classes of our own to the repository.
This adds two dummy classes that really do not depend on the schema and simply demonstrates installing classes.

In [ ]:
mof= '''
    class TST_Class1 {
          [Key]
        string InstanceID;
        string Prop1;
    };

    class TST_Class2 {
          [Key]
        string InstanceID;
        string Prop2;
    };

      [Association]
    class TST_Association12 {
          [Key]
        TST_Class1 REF Ref1;
          [Key]
        TST_Class2 REF Ref2;
    };
'''

conn.compile_mof_string(mof)
# print the class returned from the server using the __str__ magic method and __repr__ magic method
print(conn.GetClass("TST_Class1"))
print(repr(conn.GetClass("TST_Class2")))

## Adding more objects

Now that we have classes in the repository we can add some CIM Instances. The following cell defines an instance of each of the new classes and also an assoiation instance that relates the instances of the classes. 

In this case, they are defined as instances of pywbem objects to demonstrated adding pywbem objects to the repository.

In [ ]:
ns = conn.default_namespace

c1_key = pywbem.CIMProperty('InstanceID', type='string', value='111')
c1_path = pywbem.CIMInstanceName(
    'TST_Class1',
    keybindings={c1_key.name: c1_key.value},
    namespace=ns
)
c1 = pywbem.CIMInstance(
    'TST_Class1',
    properties=[
        c1_key,
        pywbem.CIMProperty('Prop1', type='string', value='1'),
    ],
    path=c1_path,
)

c2_key = pywbem.CIMProperty('InstanceID', type='string', value='222')
c2_path = pywbem.CIMInstanceName(
    'TST_Class2',
    keybindings={c2_key.name: c2_key.value},
    namespace=ns
)
c2 = pywbem.CIMInstance(
    'TST_Class2',
    properties=[
        c2_key,
        pywbem.CIMProperty('Prop2', type='string', value='2'),
    ],
    path=c2_path,
)

a12_key1 = pywbem.CIMProperty('Ref1', type='reference', value=c1_path)
a12_key2 = pywbem.CIMProperty('Ref2', type='reference', value=c2_path)
a12_path = pywbem.CIMInstanceName(
    'TST_Association12',
    keybindings={
        a12_key1.name: a12_key1.value,
        a12_key2.name: a12_key2.value,
    },
)
a12 = pywbem.CIMInstance(
    'TST_Association12',
    properties=[
        a12_key1,
        a12_key2,
    ],
    path=a12_path,
)

conn.add_cimobjects([c1, c2, a12])

returned_instances = conn.EnumerateInstances('TST_Class2')
for inst in returned_instances:
    print(inst.tomof())

## The association, associators and references can also be retrieved from the server

Note that in most cases the association can be requests just as any other instance in thhe server.

However, the actual relation between a source instance and it associated instances can be requested through the
Reference and Association methods. These are powerful methods that provide the basis for navigating through the CIM model and profiles on a server.

Pywbem_mock manages both Associations and References defined in the mock repository.

In [ ]:
returned_assoc_insts = conn.EnumerateInstances("TST_Association12")
for inst in returned_assoc_insts:
    print(inst.tomof())
    
cls_111_paths = conn.EnumerateInstanceNames('TST_Class1')
for path in cls_111_paths:
    returned_refs = conn.References(path)
    print('Refs for %s' % path)
    for inst in returned_refs:
        print(inst.tomof())
        
# and the associators operations

for path in cls_111_paths:
    returned_assocs = conn.Associators(path)
    print('Associations for %s' % path)
    for inst in returned_assocs:
        print(inst.tomof())
    for inst in returned_assocs:
        print('Returned association path: %s:' % inst.path)
    

TODO; Future
We want to expand this notebook to demonstrate:

1. Adding associations to the repository through mof since this requires a specific feature in mof, the alias
2. Demonstrate removing our instances